In [1]:
import sklearn.metrics
import pandas as pd

In [7]:
!pip install -U spacy

In [13]:
!python -m spacy download en_core_web_sm

  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.0-cp37-none-any.whl size=12019129 sha256=a3b034633201162dd9ca23bf8613b55154c46f8acd4c7ebb38e4acacbf877daa
  Stored in directory: C:\Users\Sayan\AppData\Local\Temp\pip-ephem-wheel-cache-irvm70l5\wheels\48\5c\1c\15f9d02afc8221a668d2172446dd8467b20cdb9aef80a172a4
Successfully built en-core-web-sm
[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [ ]:
!python -m spacy download en_core_web_lg

In [2]:
import spacy
from spacy.lang.en.examples import sentences
from spacy.symbols import  LEMMA # POS, TAG, ORTH, add'l if desired
import pickle

In [41]:
import pickle

import keras

from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, TensorBoard
from keras import optimizers
from keras.layers import Dense, recurrent, GRU, Bidirectional #Dropout, Flatten, RNN, SimpleRNN,
from keras.layers import Embedding, GlobalMaxPooling1D, LSTM #Conv2D, MaxPooling2D, Conv1D,
from keras import backend as K
from keras.layers import Flatten
from keras.preprocessing import text

import numpy as np

In [47]:
def preprocessing(data):
    lemma = []
    text = data['tweet']

    nlp = spacy.load('en_core_web_sm') #might have to download
    doc = nlp("This is a sentence.")
    print(doc)
    #Marks -PRON- for Pronoun
    for tweet in text:
        tweet = nlp(tweet)
        lemma.append([])
        for token in tweet:
            # if token != "@USER":
#             print(token)
            lemma[-1].append(token.lemma_)
#         print(lemma)
#         print("------------------------")

    indexed = []

    max_len = len(max(lemma, key=len))
    print(max_len)
    #this is if you haven't saved the data and need to re-run it
#     pickle.dump(lemma, open('lemma_tokens.pkl', 'wb'))



    #to create vocabulary
#     Here each word is assigned a value in ascending order
    vocabulary = {}
    value = 1
    for tweet in lemma:
        for token in tweet:
            if token not in vocabulary:
                vocabulary[token] = value
                value += 1

#     print(vocabulary)

#     #this is if you haven't saved the data and need to re-run it
#     pickle.dump(vocabulary, open('vocabulary.pkl', 'wb'))

#     # previously created dictionary of vocabulary
#     vocabulary = pickle.load(open('vocabulary.pkl', 'rb'))

    #turn tokenized tweets into integers
    for tweet in lemma:
        row = []
        for token in tweet:
            if token in vocabulary: #this ignores new vocabulary from test or dev
                row.append(vocabulary[token]) #is extend better here?
            #possibly have to add an else here to add the word to vocabulary dictionary
            #and then run the rest of it for new words from dev or test
        while len(row) < max_len:
            row.append(0)
        indexed.append(row)

    inputs = np.array(indexed)
    print(inputs.shape)
    print(inputs[:10])
#         #this is if you haven't saved the data and need to re-run it
#     pickle.dump(indexed, open('lemma_indexed.pkl', 'wb'))

#     # won't work right, change to whatever column header is
    outputs = data.loc[: , 'subtask_a']
    print(outputs[:20])
    outputs = outputs.values.copy()
    print(outputs[:20])

    return inputs, outputs, vocabulary, max_len

In [50]:
def train_and_predict(train_data: pd.DataFrame,
                      dev_data: pd.DataFrame) -> pd.DataFrame:

    print("Training Starts")
    #this is if you haven't saved the data and need to re-run it
    train_inputs, train_outputs, vocabulary, max_len = preprocessing(train_data)


#     train_inputs = np.array(pickle.load(open('lemma_indexed.pkl', 'rb')))
    print("train_inputs")
    print(train_inputs[:10])
    print(train_inputs.shape)

#     train_outputs = train_inputs.loc[:, 'subtask_a'].values.copy() #train_data
    print("train_outputs")
    print(train_outputs[:10])
    print(train_outputs.shape)
#     vocabulary = pickle.load(open('vocabulary.pkl', 'rb')) #vocab above but this is better version?
#     print(vocabulary)

    #play with more layers
    rnn = Sequential()
    rnn.add(Embedding(input_dim = len(vocabulary) +1, output_dim = 75, input_length=max_len))
    rnn.add(Flatten())
    rnn.add(Dense(1, activation='sigmoid'))
    # compile the model
    rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # summarize the model
    print(rnn.summary())
#     rnn.add(Bidirectional(GRU(128, activation='tanh', return_sequences=True)))
#     rnn.add(GlobalMaxPooling1D())
#     rnn.add(Dense(1, activation='sigmoid'))
#     rnn.compile(loss='binary_crossentropy',
#              optimizer= 'adam', metrics=['accuracy'])


    rnn.fit(x = train_inputs, y = train_outputs, epochs=10, verbose=0)
    loss, accuracy = rnn.evaluate(train_inputs, train_outputs, verbose=0)
    print('Training Accuracy: %f' % (accuracy*100))

#     dev_inputs, dev_outputs, vocabulary, max_len = preprocessing(dev_data)
#     predictions = np.round(rnn.predict(dev_inputs)).astype(int)


#     dev_predictions = dev_data.copy()

#     dev_predictions[offense] = predictions
#     print("devpredict")
#     print(dev_predictions)

#     return dev_predictions

In [51]:
def main():

    # reads train and dev data into Pandas data frames
#     read_csv_kwargs = dict(sep="\t",
#                            converters={e: offense_to_int.get for e in offense})
#     train_and_dev = pd.read_csv("offenseval-training-v1.tsv", **read_csv_kwargs)
#     len_of_data = len(train_and_dev)


#     #possibly better way to split
# #     from sklearn.model_selection import train_test_split
# # train, test = train_test_split(df, test_size=0.33, random_state=42)
#     dev_len = int(len_of_data/4)
#     train_data = train_and_dev[:dev_len]
#     print(train_data.head)
#     dev_data = train_and_dev[dev_len:]
#     print(dev_data.head)
    train_data = pd.read_csv('./training-v1/offenseval-training-v1.tsv', sep='\t', header=0)
    col_Names=["tweet", "subtask_a", "subtask_b", "subtask_c"]
    dev_data = pd.read_csv('./trial-data/offenseval-trial.txt', sep='\t', names=col_Names)
    cleanup_nums = {"subtask_a":     {"OFF": 1, "NOT": 0}}
    train_data.replace(cleanup_nums, inplace=True)
    dev_data.replace(cleanup_nums, inplace=True)
#     preprocessing(train_data)
#     makes predictions on the dev set
    dev_predictions = train_and_predict(train_data, dev_data)
#     print("dev_predictions")
#     print(dev_predictions[:10])
#     print("type: " + str(type(dev_predictions)))

#     dev_predictions_formatted = dev_predictions


# #     The prediction format is a comma-delimited text file with a *.csv extension which
# #     should contain two columns: (1) the sample ID, and (2) the sample label.
# #      You can name the file anything you like, so long as it has a .csv extension.
# #      The order of the entries is not important, but there must be one prediction for each ID.
# #       The CSV file should not have a header row.

# # saves predictions and prints out multi-label accuracy
# #     index = ids?

#     dev_predictions = pd.DataFrame(dev_predictions)
#     dev_predictions = dev_predictions.loc[:, ['id', 'tweet', 'subtask_a']]
#     dev_predictions['gold_label'] = dev_data[offense]

#     dev_predictions.to_csv("error_analysis.csv", index=False, header= True) #, sep="\t"
#     print("accuracy: {:.3f}".format(sklearn.metrics.jaccard_similarity_score(
#         dev_data[offense], dev_predictions[offense])))




if __name__ == "__main__": #makes it easier to pull these in from elsewhere b/c name == main only here
    main()

Training Starts
This is a sentence.
112
(13240, 112)
[[  1   2   3 ...   0   0   0]
 [  1   1  15 ...   0   0   0]
 [ 27  13  28 ...   0   0   0]
 ...
 [  1  13  77 ...   0   0   0]
 [  1  99 100 ...   0   0   0]
 [  1 102 103 ...   0   0   0]]
0     1
1     1
2     0
3     1
4     0
5     1
6     1
7     1
8     0
9     1
10    0
11    0
12    1
13    0
14    0
15    0
16    0
17    0
18    0
19    1
Name: subtask_a, dtype: int64
[1 1 0 1 0 1 1 1 0 1 0 0 1 0 0 0 0 0 0 1]
train_inputs
[[  1   2   3 ...   0   0   0]
 [  1   1  15 ...   0   0   0]
 [ 27  13  28 ...   0   0   0]
 ...
 [  1  13  77 ...   0   0   0]
 [  1  99 100 ...   0   0   0]
 [  1 102 103 ...   0   0   0]]
(13240, 112)
train_outputs
[1 1 0 1 0 1 1 1 0 1]
(13240,)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 112, 75)           1539675   
______________________________________________________________